# Eksempel, hvordan splitte vegobjekt i CSV 

Tusen takk for flott demo! 

Her er tilbakemelding på hvordan jeg mener CSV-eksporten bør håndtere objekter hvor det ikke er mulig å gjengi all informasjon på en og kun en rad.

I dag gjøres det en permutasjon hvor det blir en rad per lokasjon/vegsystemreferanse - element. For NVDB objekt https://apilesv3.utv.atlas.vegvesen.no/vegobjekter/67/460557991/8.json?dybde=full 
ser dette slik ut: 


| VEGOBJEKT-ID | VEGSYSTEMREFERANSE  | STEDFESTING  | GEOMETRI  | 
|---|---|---|---|
| 460557991  | RV706 S2D1 m2638-2653 | 2394247 0 1  |  _Hele geometrien_  | 
| 460557991  | RV706 S2D1 m3625-5276  | 2394247 0 1 |  _Hele geometrien_  |
| 460557991  | RV706 S2D1 m2653-3570 | 2394247 0 1  |  _Hele geometrien_  | 
|  460557991 | RV706 S2D1 m1812 KD4 m536-745 | 2394247 0 1 | _Hele geometrien_  |





Det jeg misliker - er at geometri og stedfesting er identiske for hver av disse radene. I stedet ønsker jeg at det er 1:1 forhold mellom vegsystemreferanse, stedfesting og geometri når det ikke lenger er mulig å gjengi all informasjon om vegobjektet på en rad i CSV-eksport. 

Dette ønsket er sterkt inspirert av hvordan jeg er vant med at NVDB opptrer i V2, spesifikt med bruk av parameteren ```inkluder=vegsegmenter``` (eller helst ```inkluder=alle```, som jeg pleier bruke. 

Underveis har jeg avdekket litt snål oppførsel fra NVDB api V3 som jeg tror det er verdt å titte nærmere på. 

### Vegsegmenter-oppdeling av NVDB objekt fra V2 


In [14]:
import requests
import json
import hvplot 
import pandas as pd

# Geodata-stuff 
import geopandas as gpd
from shapely import wkt
import hvplot.pandas
from holoviews import opts
import holoviews as hv
import cartopy.crs as ccrs

# støtte for scroll med mus
opts.defaults( 
    opts.Overlay(
        width=500, height=600,
        active_tools=['wheel_zoom']
))

In [2]:
r = requests.get( 'https://www.vegvesen.no/nvdb/api/v2/vegobjekter/67/460557991.json?&inkluder=vegsegmenter,lokasjon')
tun_v2 = r.json()

In [3]:
for seg in tun_v2['vegsegmenter']:
    seg['vegref'] = seg['vegreferanse']['kortform']
    seg['geometry'] = wkt.loads( seg['geometri']['wkt'] )
    seg['veglenkeid'] = seg['stedfesting']['veglenkeid']
    seg['fra_posisjon'] = seg['stedfesting']['fra_posisjon']
    seg['til_posisjon'] = seg['stedfesting']['til_posisjon']

In [4]:
v2 = pd.DataFrame( tun_v2['vegsegmenter'])
v2[['veglenkeid', 'fra_posisjon', 'til_posisjon', 'strekningslengde', 'vegref', 'geometry']]

,veglenkeid,fra_posisjon,til_posisjon,strekningslengde,vegref,geometry
0,2394247,0.000000,1.000000,932,5000 Rv706 hp2 m10000-10932,LINESTRING Z (273619.78149 7042013.35913 32.95...
1,2394246,0.000000,0.967919,1651,5000 Rv706 hp2 m10932-12583,LINESTRING Z (272743.13196 7042085.05811 12.47...
2,2394251,0.719539,1.000000,209,5000 Rv706 hp80 m21536-21745,LINESTRING Z (272940.43617 7042150.50986 15.55...


In [5]:
v2_gdf = gpd.GeoDataFrame( v2,  geometry = 'geometry')

In [6]:
v2_gdf.hvplot( crs=ccrs.UTM(zone=33), c='vegref', cmap='Category10', tiles='OSM',
    hover_cols=['veglenkeid', 'fra_posisjon', 'til_posisjon','strekningslengde' ,'vegref'], 
    line_width=5)

:Overlay
   .Tiles.I :Tiles   [x,y]
   .Path.I  :Path   [Longitude,Latitude]   (vegref,veglenkeid,fra_posisjon,til_posisjon,strekningslengde)

_**Tips**: Musehjul funker knallbra for zoom. Ellers er det zoomverktøy som kan aktiveres/deaktiveres i høyre marg._

Her er objektet pent og pyntelig delt opp i tre pene biter, med 1:1 korrespondanse mellom stedfestingen, vegreferanse og geometri. Dette har jeg brukt MYE, blant annet i mitt python-bibliotek med QGIS-integrasjon.

Merk at i CSV-eksporten fra V2 blir de to første bitene slått sammen til én, fordi vegreferanseverdiene matcher (meterverdiene m10000-10932 og m10932-12583 kan forenkles til m10000-12583).

### Vegsegmenter-oppdeling av NVDB objekt fra V3

Vi gjentar øvelsen, men nå fra NVDB api V3

In [7]:
from copy import deepcopy 
r = requests.get( 'https://apilesv3.utv.atlas.vegvesen.no/vegobjekter/67/460557991/8.json?&inkluder=vegsegmenter,lokasjon')
tun_v3 = r.json()

data = []
for seg3 in tun_v3['vegsegmenter']:
    nyseg = deepcopy( seg3)
    nyseg['vegref'] = nyseg['vegsystemreferanse']['kortform']
    nyseg['geometry'] = wkt.loads( nyseg['geometri']['wkt'] ) 
    data.append( deepcopy(nyseg))

df_v3 = pd.DataFrame( data ) 

In [8]:
df_v3[['vegref', 'lengde', 'veglenkesekvensid', 
    'startposisjon', 'sluttposisjon', 'geometry']].sort_values(by=['veglenkesekvensid', 'startposisjon'])

,vegref,lengde,veglenkesekvensid,startposisjon,sluttposisjon,geometry
2,RV706,1651.225,2394246,0.000000,0.967919,"LINESTRING Z (272743.132 7042085.058 12.479, 2..."
6,RV706,1651.225,2394246,0.000000,0.967919,"LINESTRING Z (272743.132 7042085.058 12.479, 2..."
10,RV706,1651.225,2394246,0.000000,0.967919,"LINESTRING Z (272743.132 7042085.058 12.479, 2..."
14,RV706 S2D1 m3625-5276,1651.225,2394246,0.000000,0.967919,"LINESTRING Z (272743.132 7042085.058 12.479, 2..."
18,RV706 S2D1 m3625-5276,1651.225,2394246,0.000000,0.967919,"LINESTRING Z (272743.132 7042085.058 12.479, 2..."
22,RV706 S2D1 m3625-5276,1651.225,2394246,0.000000,0.967919,"LINESTRING Z (272743.132 7042085.058 12.479, 2..."
26,RV706 S2D1 m3625-5276,1651.225,2394246,0.000000,0.967919,"LINESTRING Z (272743.132 7042085.058 12.479, 2..."
30,RV706 S2D1 m3625-5276,1651.225,2394246,0.000000,0.967919,"LINESTRING Z (272743.132 7042085.058 12.479, 2..."
33,RV706 S2D1 m3625-5276,1651.225,2394246,0.000000,0.967919,"LINESTRING Z (272743.132 7042085.058 12.479, 2..."
0,RV706,14.938,2394247,0.000000,0.016029,"LINESTRING Z (273619.781 7042013.359 32.955, 2..."


Dette ser litt ... rart .. ut. Én ting er at vi har 36 vegsegmenter der NVDB api V2 har tre. Men det ser også ut til at alle  vegsegmenter gjentas minst én gang: 

In [9]:
df_v3[ df_v3.duplicated(['veglenkesekvensid', 'startposisjon'], keep=False)][['vegref', 'veglenkesekvensid', 'startposisjon', 'sluttposisjon']]

,vegref,veglenkesekvensid,startposisjon,sluttposisjon
0,RV706,2394247,0.000000,0.016029
1,RV706,2394247,0.016029,1.000000
2,RV706,2394246,0.000000,0.967919
3,RV706,2394251,0.719539,1.000000
4,RV706,2394247,0.000000,0.016029
5,RV706,2394247,0.016029,1.000000
6,RV706,2394246,0.000000,0.967919
7,RV706,2394251,0.719539,1.000000
8,RV706,2394247,0.000000,0.016029
9,RV706,2394247,0.016029,1.000000


Vi fjerner duplikater og går videre: 

In [10]:
df3_clean = df_v3.drop_duplicates( subset=['vegref','veglenkesekvensid', 'startposisjon'] ).copy()
df3_clean[['vegref', 'lengde', 'veglenkesekvensid', 
    'startposisjon', 'sluttposisjon', 'geometry']].sort_values(by=['veglenkesekvensid', 'startposisjon'])

,vegref,lengde,veglenkesekvensid,startposisjon,sluttposisjon,geometry
2,RV706,1651.225,2394246,0.000000,0.967919,"LINESTRING Z (272743.132 7042085.058 12.479, 2..."
14,RV706 S2D1 m3625-5276,1651.225,2394246,0.000000,0.967919,"LINESTRING Z (272743.132 7042085.058 12.479, 2..."
0,RV706,14.938,2394247,0.000000,0.016029,"LINESTRING Z (273619.781 7042013.359 32.955, 2..."
13,RV706 S2D1 m2638-2653,14.938,2394247,0.000000,0.016029,"LINESTRING Z (273619.781 7042013.359 32.955, 2..."
1,RV706,916.968,2394247,0.016029,1.000000,"LINESTRING Z (273610.881 7042025.355 32.823, 2..."
15,RV706 S2D1 m2653-3570,916.968,2394247,0.016029,1.000000,"LINESTRING Z (273610.881 7042025.355 32.823, 2..."
3,RV706,209.048,2394251,0.719539,1.000000,"LINESTRING Z (272940.436 7042150.51 15.554, 27..."
35,RV706 S2D1 m1812 KD4 m536-745,209.048,2394251,0.719539,1.000000,"LINESTRING Z (272940.436 7042150.51 15.554, 27..."


Hmm.... nå finner vi iallfall en viss systematikk: Vi har parvis en "Rv706" og en fullverdig vegsystemreferanse. 

Vi fjerner de bitene med minst informasjon (der det kun står ```RV706```): 

In [11]:
df3_clean2 = df3_clean[  df3_clean['vegref'] != 'RV706' ].copy()
df3_clean2[['vegref', 'lengde', 'veglenkesekvensid', 
    'startposisjon', 'sluttposisjon', 'geometry']].sort_values(by=['veglenkesekvensid', 'startposisjon'])

,vegref,lengde,veglenkesekvensid,startposisjon,sluttposisjon,geometry
14,RV706 S2D1 m3625-5276,1651.225,2394246,0.000000,0.967919,"LINESTRING Z (272743.132 7042085.058 12.479, 2..."
13,RV706 S2D1 m2638-2653,14.938,2394247,0.000000,0.016029,"LINESTRING Z (273619.781 7042013.359 32.955, 2..."
15,RV706 S2D1 m2653-3570,916.968,2394247,0.016029,1.000000,"LINESTRING Z (273610.881 7042025.355 32.823, 2..."
35,RV706 S2D1 m1812 KD4 m536-745,209.048,2394251,0.719539,1.000000,"LINESTRING Z (272940.436 7042150.51 15.554, 27..."


## Nå snakker vi - der fikk vi skikk på vegsegmenter fra NVDB api V3

Etter litt duplikat-filtrering står vi igjen med 4 segmenter for vegobjekt [460557991](https://apilesv3.utv.atlas.vegvesen.no/vegobjekter/67/460557991/8.json?&inkluder=vegsegmenter,lokasjon'), med et 1:1 forhold mellom vegsystemreferanse, stedfesting og geometri. **Dette er den oppdelingen jeg ønsker i CSV - eksporten**. 

In [12]:
gdf_v3 = gpd.GeoDataFrame( df3_clean2,  geometry = 'geometry')
gdf_v3.hvplot( crs=ccrs.UTM(zone=33), c='vegref', cmap='Category10', tiles='OSM',
    hover_cols=['veglenkeid', 'fra_posisjon', 'til_posisjon','strekningslengde' ,'vegref'], 
    line_width=5)

:Overlay
   .Tiles.I :Tiles   [x,y]
   .Path.I  :Path   [Longitude,Latitude]   (vegref)

In [16]:
gdf_v3.hvplot( crs=ccrs.UTM(zone=33), c='vegref', cmap='Category10', tiles='OSM', line_width=5, legend=False) +\
v2_gdf.hvplot( crs=ccrs.UTM(zone=33), c='vegref', cmap='Category10', tiles='OSM', line_width=5, legend=False)

:Layout
   .Overlay.I  :Overlay
      .Tiles.I :Tiles   [x,y]
      .Path.I  :Path   [Longitude,Latitude]   (vegref)
   .Overlay.II :Overlay
      .Tiles.I :Tiles   [x,y]
      .Path.I  :Path   [Longitude,Latitude]   (vegref)